In [1]:
import pandas as pd
import glob

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
%pip install --quiet optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
import optuna
import os

/home/duarbrun/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
optuna.__version__

'4.2.1'

## TABU SEARCH

In [6]:
files = ['4-FullIns_4', 'ash331GPIA', 'DSJC250.1', 'le450_15a', 'qg.order30', 'queen13_13']
columns_tabu = ['conflitos', 'tempo_us', 'iter_found_best', 
           'max_iter', 'T_iter', 'num_color', 'filename', 
           'num_vertices', 'num_edges', 'init_method' ]

In [ ]:
dfs = []

for file in files:
    df = pd.read_csv(f'results/tabu/analysis/{file}.txt', sep=';', names=columns_tabu, header=None)
    df['conflitos_norm'] = (df["conflitos"] - df["conflitos"].min()) / (df["conflitos"].max() - df["conflitos"].min() if df["conflitos"].max()!= df["conflitos"].min() else 0)
    df['tempo_norm'] = (df["tempo_us"] - df["tempo_us"].min()) / (df["tempo_us"].max() - df["tempo_us"].min() if df["tempo_us"].max()!= df["tempo_us"].min() else 0)
    dfs.append(df)

df_tabu = pd.concat(dfs, ignore_index=True)
df_tabu.tail()

,conflitos,tempo_us,iter_found_best,max_iter,T_iter,num_color,filename,num_vertices,num_edges,init_method,conflitos_norm,tempo_norm
5755,60,11048850,163,10000,100,13,instances/queen13_13.col,169,6656,random,0.21875,0.755890
5756,62,11650160,262,10000,100,13,instances/queen13_13.col,169,6656,random,0.25000,0.797101
5757,64,14143315,188,10000,100,13,instances/queen13_13.col,169,6656,random,0.28125,0.967969
5758,56,11127899,177,10000,100,13,instances/queen13_13.col,169,6656,random,0.15625,0.761307
5759,60,12387752,149,10000,100,13,instances/queen13_13.col,169,6656,random,0.21875,0.847651


In [14]:
df_tabu['conflitos_norm'] = (df_tabu["conflitos"] - df_tabu["conflitos"].min()) / (df_tabu["conflitos"].max() - df_tabu["conflitos"].min() if df_tabu["conflitos"].max()!= df_tabu["conflitos"].min() else 0)
df_tabu['tempo_norm'] = (df_tabu["tempo_us"] - df_tabu["tempo_us"].min()) / (df_tabu["tempo_us"].max() - df_tabu["tempo_us"].min() if df_tabu["tempo_us"].max()!= df_tabu["tempo_us"].min() else 0)
def objective_v4(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    T_iter = trial.suggest_categorical("T_iter", df_tabu["T_iter"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df_tabu["max_iter"].unique().tolist())
    init_method = trial.suggest_categorical("init_method", df_tabu["init_method"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_tabu[(df_tabu["T_iter"] == T_iter) &
                (df_tabu["max_iter"] == max_iter) &
                (df_tabu["init_method"] == init_method)]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor


In [22]:
n_trials = df_tabu[["T_iter", "max_iter", "init_method"]].drop_duplicates().shape[0]

print(n_trials)

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v4, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-04-07 15:23:07,200] A new study created in memory with name: no-name-21bd01fc-3c4a-49be-9390-c768fab0ff96
[I 2025-04-07 15:23:07,211] Trial 0 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,215] Trial 1 finished with value: 0.24643939393939393 and parameters: {'T_iter': 25, 'max_iter': 10000, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,219] Trial 2 finished with value: 0.3259848484848485 and parameters: {'T_iter': 10, 'max_iter': 100, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,223] Trial 3 finished with value: 0.3712878787878787 and parameters: {'T_iter': 5, 'max_iter': 1000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,228] Trial 4 finished with value: 0.3156060

48


[I 2025-04-07 15:23:07,407] Trial 31 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,414] Trial 32 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,421] Trial 33 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,429] Trial 34 finished with value: 0.3156060606060606 and parameters: {'T_iter': 10, 'max_iter': 10000, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,437] Trial 35 finished with value: 0.2727272727272727 and parameters: {'T_iter': 75, 'max_iter': 100, 'init_method': 'pseudogreedy_v3'}. Be

Melhores parâmetros encontrados:
{'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}


## SIMULATED ANNEALING

In [1]:
files = ['4-FullIns_4', 'ash331GPIA', 'DSJC250.1', 'le450_15a', 'qg.order30', 'queen13_13']
columns_sa = ['conflitos', 'tempo_us', 'T_max', 'T_min', 
                'max_iter', 'max_worsen', 'alpha', 'iter_found_best',
                'curr_Temp', 'filename', 'num_vertices', 'num_edges', 'init_method' ]

In [ ]:
dfs = []

for file in files:
    df = pd.read_csv(f'results/sa/analysis/{file}.txt', sep=';', names=columns_sa, header=None)
    df['conflitos_norm'] = (df["conflitos"] - df["conflitos"].min()) / (df["conflitos"].max() - df["conflitos"].min() if df["conflitos"].max()!= df["conflitos"].min() else 0)
    df['tempo_norm'] = (df["tempo_us"] - df["tempo_us"].min()) / (df["tempo_us"].max() - df["tempo_us"].min() if df["tempo_us"].max()!= df["tempo_us"].min() else 0)
    dfs.append(df)

df_sa = pd.concat(dfs, ignore_index=True)
df_sa.head()

,conflitos,tempo_us,T_max,T_min,max_iter,max_worsen,alpha,iter_found_best,curr_Temp,filename,num_vertices,num_edges,init_method,conflitos_norm,tempo_norm
0,636,850800,100,10,1000,1000,0.6,7354,7,instances/4-FullIns_4.col,690,6650,random,0.925743,0.002494
1,616,898268,100,10,1000,1000,0.6,8085,7,instances/4-FullIns_4.col,690,6650,random,0.892739,0.006902
2,612,895010,100,10,1000,1000,0.6,7669,7,instances/4-FullIns_4.col,690,6650,random,0.886139,0.006600
3,649,885161,100,10,1000,1000,0.6,5471,7,instances/4-FullIns_4.col,690,6650,random,0.947195,0.005685
4,635,841120,100,10,1000,1000,0.6,8415,7,instances/4-FullIns_4.col,690,6650,random,0.924092,0.001595


In [37]:
df_sa['conflitos_norm'] = (df_sa["conflitos"] - df_sa["conflitos"].min()) / (df_sa["conflitos"].max() - df_sa["conflitos"].min() if df_sa["conflitos"].max()!= df_sa["conflitos"].min() else 0)
df_sa['tempo_norm'] = (df_sa["tempo_us"] - df_sa["tempo_us"].min()) / (df_sa["tempo_us"].max() - df_sa["tempo_us"].min() if df_sa["tempo_us"].max()!= df_sa["tempo_us"].min() else 0)
df_sa[df_sa['init_method']=='random']
def objective_v5(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    T_max = trial.suggest_categorical("T_max", df_sa["T_max"].unique().tolist())
    alpha = trial.suggest_categorical("alpha", df_sa["alpha"].unique().tolist())
    # init_method = trial.suggest_categorical("init_method", df_sa["init_method"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_sa[(df_sa["T_max"] == T_max) &
                (df_sa["alpha"] == alpha) ]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor


In [38]:
n_trials = df_sa[["T_max", "alpha", "filename"]].drop_duplicates().shape[0]

print(n_trials)

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v5, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-04-20 23:19:01,489] A new study created in memory with name: no-name-ac949949-400a-4d52-a720-dbc591a0bbea
[I 2025-04-20 23:19:01,492] Trial 0 finished with value: 0.3453586497890295 and parameters: {'T_max': 1000, 'alpha': 0.8}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,493] Trial 1 finished with value: 0.35037505860290674 and parameters: {'T_max': 10000, 'alpha': 0.7}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,496] Trial 2 finished with value: 0.3469936708860759 and parameters: {'T_max': 100, 'alpha': 0.7}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,497] Trial 3 finished with value: 0.3472222222222222 and parameters: {'T_max': 5000, 'alpha': 0.8}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,499] Trial 4 finished with value: 0.34172526957337085 and parameters: {'T_max': 10000, 'alpha': 0.9}. Best is trial 4 with value: 0.34172526957337085.
[I 2025-04-20 23:19:01,501]

96


[I 2025-04-20 23:19:01,692] Trial 83 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,695] Trial 84 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,698] Trial 85 finished with value: 0.34172526957337085 and parameters: {'T_max': 10000, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,700] Trial 86 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,702] Trial 87 finished with value: 0.34478434130332863 and parameters: {'T_max': 100, 'alpha': 0.8}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,705] Trial 88 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is t

Melhores parâmetros encontrados:
{'T_max': 100, 'alpha': 0.9}
